# \__getatrr__ (self,attr)

\_getattr__是python里的一个内建函数，可以很方便地动态返回一个属性；
当调用不存在的属性时，Python会试图调用__getattr__(self,attr)来获取属性，并且返回；

正常情况下，当我们调用类的方法或属性时，如果不存在，就会报错。比如定义Student类：调用name属性，没问题，但是，调用不存在的score属性，就有问题了：错误信息很清楚地告诉我们，没有找到score这个attribute。

In [1]:
class Student(object):
    def __init__(self):
        self.name = 'Michael'

s = Student()
print(s.name)
print(s.score)

Michael


AttributeError: 'Student' object has no attribute 'score'

要避免这个错误，除了可以加上一个score属性外，Python还有另一个机制，那就是写一个\__getattr__()方法，动态返回一个属性。

当调用不存在的属性时，比如score，Python解释器会试图调用__getattr__(self, 'score')来尝试获得属性，这样，我们就有机会返回score的值：

In [2]:
class Student(object):
    def __init__(self):
        self.name = 'Michael'
    def __getattr__(self,attr):
        if attr == 'score':
            return 99
    
s = Student()
print(s.name)
print(s.score)

Michael
99


返回函数也可以的，只是调用方法要改变。

In [9]:
class Student(object):
    def __getattr__(self,attr):
        if attr == 'age':
            return lambda:25
    
s = Student()
print(s.age())
print(s.abc)

25
None


**注意，只有在没有找到属性的情况下，才调用__getattr__，已有的属性，比如name，不会在__getattr__中查找。**

此外，注意到任意调用如s.abc都会返回None，这是因为我们定义的__getattr__默认返回就是None。

要让class只响应特定的几个属性，我们就要按照约定，抛出AttributeError的错误：

In [11]:
class Student(object):
    def __getattr__(self,attr):
        if attr == 'age':
            return 28
        raise AttributeError("'Student' object has no attribute '{}'".format(attr))
    
s = Student()
print(s.age)
print(s.abc)

28


AttributeError: 'Student' object has no attribute 'abc'

这实际上可以把一个类的所有属性和方法调用全部动态化处理了，不需要任何特殊手段。

这种完全动态调用的特性有什么实际作用呢？作用就是，可以针对完全动态的情况作调用。

举个例子：

现在很多网站都搞REST API，比如新浪微博、豆瓣啥的，调用API的URL类似：

http://api.server/user/friends

http://api.server/user/timeline/list

如果要写SDK，给每个URL对应的API都写一个方法，那得累死，而且，API一旦改动，SDK也要改。

利用完全动态的__getattr__，我们可以写出一个链式调用：

In [17]:
class Chain(object):
    def __init__(self,path=''):
        self._path = path
    def __getattr__(self,attr):
        return Chain('{}/{}'.format(self._path,attr))
    def __str__(self):
        return self._path
    __repr__=__str__
    
print(Chain().status)  
print(Chain().status.user.timeline.list)

/status
/status/user/timeline/list


实例 Chain()获取 status 属性，由于没有找该属性，调用\__getattr__， 其中 self._path 是空，attr 是 status 因而返回 Chain('/status'), 参数'/status'被传递给 path，赋值给self._path.


Chain().status.user.timeline.list “敲回车键”“()”和“.”和“回车键”都是从左向右的顺序运算符，类比成数学中的加和减。
步骤：

1、Chain是类名，对它进行“()”运算，即调用__init__(self)(这个有讲过哦),会生成一个实例c1，c1=Chain(path='')。

2、对实例c1进行“.”运算，增加一个“status”属性，即调用__getattr__(self, status),返回
一个新实例c2,c2=Chain(path='/status')。

3、对实例c2进行“.”运算，增加一个“user”属性，即调用__getattr__(self, user),返回
一个新实例c3,c3=Chain(path='/status/user')。

4、对实例c3进行“.”运算，增加一个“timeline”属性，即调用__getattr__(self, timeline),返回
一个新实例c4,c4=Chain(path='/status/user/timeline')。

5、对实例c4进行“.”运算，增加一个“list”属性，即调用__getattr__(self, list),返回
一个新实例c5,c5=Chain(path='/status/user/timeline/list')。

6、对实例c5进行“回车键”运算，即调用__repr__(self)，返回c5._path,即输出'/status/user/timeline/list'。


这样，无论API怎么变，SDK都可以根据URL实现完全动态的调用，而且，不随API的增加而改变！

# \__call__

一个对象实例可以有自己的属性和方法，当我们调用实例方法时，我们用instance.method()来调用。能不能直接在实例本身上调用呢？在Python中，答案是肯定的。

任何类，只需要定义一个\__call__()方法，就可以直接对实例进行调用。请看示例：

In [30]:
class Student(object):
    def __init__(self,name):
        self.name = name
    def __call__(self):
        print('My name is {}'.format(self.name))
    
s = Student('Jamie')
s()

print(callable(s))
print(callable(max))
print(callable(1))
print(callable('str'))

My name is Jamie
True
True
False
False


\__call__()还可以定义参数。对实例进行直接调用就好比对一个函数进行调用一样，所以你完全可以把对象看成函数，把函数看成对象，因为这两者之间本来就没啥根本的区别。

如果你把对象看成函数，那么函数本身其实也可以在运行期动态创建出来，因为类的实例都是运行期创建出来的，这么一来，我们就模糊了对象和函数的界限。

那么，怎么判断一个变量是对象还是函数呢？其实，更多的时候，我们需要判断一个对象是否能被调用，能被调用的对象就是一个Callable对象，比如函数和我们上面定义的带有\__call__()的类实例


还有些REST API会把参数放到URL中，比如GitHub的API：

`GET /users/:user/repos`

调用时，需要把:user替换为实际用户名。如果我们能写出这样的链式调用：

`Chain().users('michael').repos`

In [33]:
class Chain(object):
    def __init__(self,path=''):
        self._path = path
    def __getattr__(self,attr):
        return Chain('{}/{}'.format(self._path,attr))
    def __call__(self, param):
        return Chain('{}/:{}'.format(self._path,param))
    
    def __str__(self):
        return self._path
    
    __repr__=__str__
    
print(Chain().status)  
print(Chain().status.user.timeline.list)
print(Chain().users('michael').repos)

/status
/status/user/timeline/list
/users/:michael/repos
